<h2>Clasificador bayesiano ingenuo de Bernoulli</h2>

<p>En esta libreta programaremos un clasificador bayesiano ingenuo en el cual se presupone que la distribución de los atributos dada la clase es una Bernoulli y hay dos posibles clases.</p>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.naive_bayes import BernoulliNB as SKBNB
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

<h3>Carga de datos</h3>

<p>El conjunto de entrenamiento consiste de 11 documentos que pertenecen a las clases de deportes (0) o informática (1). Cada documento es un vector de 8 dimensiones donde cada dimensión representa la frecuencia de las palabras en nuestro vocabulario de interés:</p>

$V =
\begin{equation}
\begin{vmatrix}
p_1 & = & gol\\
p_2 & = & maestro\\
p_3 & = & velocidad\\
p_4 & = & defensa\\
p_5 & = &  rendimiento\\
p_6 & = & campo\\
p_7 & = & movimiento\\
p_8 & = & ataque
\end{vmatrix}
\end{equation}$

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/gibranfp/CursoAprendizajeAutomatizado/master/data/dep_inf.csv')
df

,p1,p2,p3,p4,p5,p6,p7,p8,c
0,1,0,1,0,1,1,0,1,0
1,1,0,1,0,0,1,0,0,0
2,0,0,1,0,1,1,0,1,0
3,0,0,0,1,1,0,1,1,0
4,1,1,0,0,0,1,0,1,0
5,0,0,1,1,1,1,0,0,0
6,0,1,0,1,0,0,1,0,1
7,1,0,0,1,1,0,1,1,1
8,0,1,0,1,0,1,1,0,1
9,0,0,0,1,0,0,1,1,1


<p>Conversión a numpy</p>

In [8]:
data = df.to_numpy()
X_ent = data[:, : -1]
y_ent = data[:, -1]
X_nuevos = np.array([[0, 0, 0 , 1, 1, 1, 1, 1], [0, 1, 0, 1, 1, 0, 0, 1]])

<h3>Clasificador bayesiano ingenuo para distribución de Bernoulli</h3>

<p>Definimos una función para obtener la probabilidad de 0 o 1 dada una distribución de Bernoulli con parámetro.</p>

In [5]:
# Distribución de bernoulli
def bernoulli(x, q):
  return q**x * (1.0 - q)**(1.0 - x)

<p>Definimos una clase con el clasificador bayesiano ingenuo para atributos binarios y estimación de parámetros por máxima verosimilitud.</p>

In [6]:
bernoulli(np.array([[0, 1, 1], [1,0,0]]), np.array([0.2, 0.3, 0.4]))

array([[0.8, 0.3, 0.4],
       [0.2, 0.7, 0.6]])

In [11]:
class BernoulliNB:
    #Estima parámetros por máxima verosimilitud
    def fit(self, X, y):
        self.clases = np.unique(y) #Devuelve los elementos de y, sin repeticiones
        self.n_clases = self.clases.size
        self.n_atr = X.shape[-1]
        n = X.shape[0]
        
        self.qa = np.zeros((self.n_clases, self.n_atr)) # crea una matriz de ceros de |clases|x|atributos|
        self.qc = np.zeros((self.n_clases)) # crea un arreglo de ceros de tamaño |clases|
        for i,c in enumerate(self.clases):
            Xc = X[np.where(y == c)]
            nc = Xc.shape[0]
            cuentas = np.count_nonzero(Xc, axis = 0)
            self.qa[i, :] =  cuentas / nc
            self.qc[i] = nc / n
    
    # calcula a posteriori (proporcional) de un conjunto de datos
    def predict_proba(self, X):
        prop = np.zeros((X.shape[0], self.n_clases))
        for i in range(self.n_clases):
            prop[:, i] = np.prod(bernoulli(X, self.qa[i, :]), axis=1) * self.qc[i]
        return prop
    
    # Predice clases de conjunto de datos
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)
    
bnb = BernoulliNB()
bnb.fit(X_ent, y_ent)
y_bnb = bnb.predict(X_nuevos)
p_bnb = bnb.predict_proba(X_nuevos)

print(y_bnb)
print(p_bnb)

[1 0]
[[0.0015588  0.00279273]
 [0.00031176 0.        ]]


El método <tt>predict_proba</tt> calcula la proporcional de la probabilidad a posteriori.

$$P(x|y)\cdot P(y) \propto P(y|x)$$

Para obtener la probabilidad a posteriori es necesario dividir $P(x|y) \cdot P(y)$ entre $P(x)$

$$ P(x) = \sum_c P(x,y=c) = \sum_c P(x|y = c)P(y=c) $$

In [12]:
proba_bnb = (p_bnb / p_bnb.sum(axis=0))
print("Probabilidad a posteriori")
print(proba_bnb)

Probabilidad a posteriori
[[0.83333333 1.        ]
 [0.16666667 0.        ]]
